# Latihan 5: Object Counting dengan Scikit-Image

Pada latihan berikut, ArcGIS Notebook akan dimanfaatkan untuk melakukan object counting melalui fungsi `arcgis.raster` dan library Scikit yang tersedia pada Python. Untuk dapat menggunakan notebook ini, beberapa library berikut harus terlebih dahulu terinstall:

```
conda install scipy
conda install matplotlib
conda install scikit-image
```

### Inisiasi Modul-modul Python

In [ ]:
from arcgis.raster.functions import ndvi, stretch
from skimage import feature, color
import matplotlib.pyplot as plt

from arcgis.gis import GIS
agol = GIS()

### Penyiapan Data

In [ ]:
l8 = agol.content.search('"Multispectral Landsat"', 'Imagery Layer')[0]
l8

In [ ]:
l8lyr = l8.layers[0]

Baris kode berikut mendefinisikan extent dari citra Landsat yang akan diolah:

In [ ]:
l8lyr.extent = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},
                                         'type': 'extent',
                                         'xmax': 4296559.143733407,
                                         'xmin': 4219969.241391764,
                                         'ymax': 3522726.823081019,
                                         'ymin': 3492152.0117669892}
l8lyr

### Preprocessing Citra

Dengan menggunakan metode `arcgis.raster`, preprocessing citra dapat dilakukan sebelum klasifikasi menggunakan scikit-image. 

In [ ]:
stretch(ndvi(l8lyr), stretch_type='PercentClip', min_percent=30, max_percent=70, dra=True)
stretch()

Hasil dari analisis yang sudah dilakukan di atas kemudian disimpan dengan menggunakan fungsi pada matplotlib sebagai file baru, kemudian ditampilkan dengan fungsi plotting dari Matplotlib

In [ ]:
img = stretch(ndvi(l8lyr), stretch_type='PercentClip', min_percent=30, max_percent=70, dra=True).export_image(bbox=l8lyr.extent, bbox_sr=102100, size=[1200, 450],
                       export_format='jpeg', save_folder='.', save_file='centerpivotfarms.jpg', f='image')

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img = mpimg.imread('centerpivotfarms.jpg')

# what does it look like?
plt.imshow(img)
plt.show()

## Deteksi Blob dengan scikit-image

Blob dideteksi dengan menggunakan Scikit-image untuk kemudian diplot dengan Matplotlib. Blob dideteksi menggunakan algoritma [Difference of Gaussian (DoG)](http://scikit-image.org/docs/dev/api/skimage.feature.html#skimage.feature.blob_dog).

In [ ]:

bw = img.mean(axis=2)

fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(1,1,1)

blobs_dog = [(x[0],x[1],x[2]) for x in feature.blob_dog(-bw, 
                                                        min_sigma=4, 
                                                        max_sigma=8,
                                                        threshold=0.1,
                                                        overlap=0.6)]

#remove duplicates
blobs_dog = set(blobs_dog)

img_blobs = color.gray2rgb(img)

for blob in blobs_dog:
    y, x, r = blob
    c = plt.Circle((x, y), r+1, color='red', linewidth=2, fill=False)
    ax.add_patch(c)

plt.imshow(img_blobs)
plt.title('Pertanian Lingkar')

plt.show()
print('Jumlah pertanian lingkar terdekteksi: ' + str(len(blobs_dog)))